In [ ]:
import sys
import os
import cv2
import importlib
import torch
import numpy as np
import argparse
import yaml
from tqdm import tqdm
from PIL import Image, ImageDraw, ImageOps
import json
import nltk
import torchvision.transforms as T
import matplotlib.pyplot as plt
import torch.nn.functional as F
from scipy import signal
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
import clip
import kornia
import torchvision
import skimage.color
import random

from utils_func import *
from html_images import *

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--device', type=str, default='cuda:0')

parser.add_argument('--root_dir', type=str, default='C:/MyFiles/CondTran/finals/quant')
parser.add_argument('--log_dir', type=str, default='logs/bert_quant')
parser.add_argument('--step', type=str, default='142124')

# Load transformer
args = parser.parse_args(args=[])
os.chdir(args.root_dir)
sys.path.append(args.root_dir)
module = importlib.import_module(f'filltran.models.colorization')
args.fill_model = getattr(module, 'Colorization')
filltran = load_model(args.fill_model, args.log_dir, args.step).to(args.device).eval().requires_grad_(False)

# Sample methods

In [ ]:
def sample(x_gray, topk, prior=None):
    _, idx_gray = filltran.hybrid_vqgan.encode(None, x_gray)
    B = idx_gray.shape[0]
    sample_shape = [16, 16]
    idx_gray = idx_gray.reshape(B, 16, 16)
    rows, cols = idx_gray.shape[1:3]

    if prior == None:
        idx_color = filltran.mask_token * torch.ones([1, rows, cols]).to(idx_gray.device).long()
    else:
        idx_color = prior

    i = 0
    for r in range(16):
        for c in range(16):
            # Input color indices
            logits = filltran.coltran(idx_color, idx_gray, None, None)
            logits = logits[:, i, :]
            logits = logits.reshape(-1, logits.shape[-1])
            logits = filltran.top_k_logits(logits, topk)
            probs = F.softmax(logits, dim=-1)
            ix = torch.multinomial(probs, num_samples=1)
            idx_color[:, r, c] = ix

            i += 1

    gen = filltran.hybrid_vqgan.decode(idx_color, idx_gray)

    return gen

# Sample dataset (raw)

In [ ]:
# Sampling arguments
img_dir = 'C:\\MyFiles\\Dataset\\imagenet\\val5000\\val'
save_dir = 'C:\\Users\\lucky\\Desktop\\bert_quant'
topk = 100
num_samples = 1
img_size = [256, 256]
sample_size = [img_size[0]//16, img_size[1]//16]

os.makedirs(save_dir, exist_ok=True)

pbar = tqdm(enumerate(os.listdir(img_dir)))
for i, filename in pbar:
    if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.JPEG') or filename.endswith('.jpeg'):
        name = filename.split('.')[0]
        I_color = Image.open(os.path.join(img_dir, filename)).convert('RGB')
        I_gray = I_color.convert('L')

        x_color = preprocess(I_color, img_size).to(args.device)
        x_gray = preprocess(I_gray, img_size).to(args.device)

        for n in range(num_samples):
            gen = sample(x_gray, topk, prior=None)
            gen = output_to_pil(gen[0])
            gen_resize = color_resize(I_gray, gen)
            gen_resize.save(os.path.join(save_dir, name+'.png'))